Reference -- https://colab.research.google.com/drive/1rDAJyqfVkZR-HQGlzeMi1tNekqyTgXgy?usp=sharing#scrollTo=-BzmirBC94-d

In [1]:
# !pip install -q langchain==0.0.346 openai==1.3.7 tiktoken==0.5.2 cohere==4.37

In [2]:
# Load environment variables
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

## Chat model

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [4]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [5]:
messages = [
    SystemMessage(
        content="You are a helpful assistant."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]

In [6]:
chat(messages)

AIMessage(content='The capital of France is Paris.')

## Embedding model

In [7]:
from langchain.embeddings import OpenAIEmbeddings

In [8]:
#Initialize the model
embeddings_model = OpenAIEmbeddings()

In [9]:
#Embed a list of texts
embeddings = embeddings_model.embed_documents(
    ["Hi there!", "Oh, hello!", "What's your name?", "My friends call me world", "Hello World!"]
)

In [10]:
print(f"Number of documents embedded: {len(embeddings)}")
print(f"Dimension of each embedding: {len(embeddings[0])}")

Number of documents embedded: 5
Dimension of each embedding: 1536


## LLMChain

In [11]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

template = """"List all the colors in a rainbow"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=StrOutputParser())

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm_chain = LLMChain(prompt=prompt, llm=chat)

llm_chain.predict()

'1. Red\n2. Orange\n3. Yellow\n4. Green\n5. Blue\n6. Indigo\n7. Violet'

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = PromptTemplate.from_template("List all colors in a {item}.")

runnable = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
runnable.invoke({"item": "rainbow"})

'Red, orange, yellow, green, blue, indigo, violet'

## Sequential chain

In [13]:
from langchain.prompts import PromptTemplate

post_prompt = PromptTemplate.from_template(
    """You are a business owner. Given the theme of a post, write a social media post to share on my socials.
    
    Theme: {theme}
    Content: This is social media post based on the theme above:
    """
)

review_prompt = PromptTemplate.from_template(
    """You are an expert social media manager. Given the presented social media post, it is your job to write down a review for the post.
    
    Media Post:
    
    from a Social Media Expert:
    """
)

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

llm = ChatOpenAI(temperature=0)

chain = (
    {"post": post_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"theme": "Having a black friday sale with 50% off on everything."})

'As a social media expert, I would rate this post as lacking in substance and engagement. It is important for social media posts to provide value to the audience, whether that be through informative content, entertaining visuals, or interactive elements. This post does not offer any of these elements and therefore falls short in capturing the attention and interest of the audience. In the future, I would recommend incorporating more engaging content and calls to action to improve the overall effectiveness of the post.'

In [15]:
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0)

post_chain = post_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()

chain = {"post": post_chain} | RunnablePassthrough.assign(review=review_chain)
chain.invoke({"theme": "Having a black friday sale with 50% off on everything."})

{'post': "🎉 BLACK FRIDAY SALE ALERT 🎉 Get ready to shop 'til you drop with our amazing 50% off sale on EVERYTHING in store! Don't miss out on these incredible deals - come by and stock up on all your favorites before they're gone! #BlackFriday #Sale #ShopTilYouDrop 🛍️🎁🔥",
 'review': 'The post is well-crafted and engaging, with a clear call to action. The use of visuals is eye-catching and relevant to the content being promoted. The caption is concise and informative, providing value to the audience. Overall, this post effectively showcases the expertise of the social media manager behind it.'}